In [2]:
import sys
sys.path.append("../")

In [3]:
import torch
import torch.nn as nn


c:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import utils.datasets as d
dataloaders=d.load_and_preprocess("sup",batch_size=20)

Processing supervised set


  0%|          | 0/20 [00:00<?, ?it/s]c:\Users\micha\OneDrive\Plocha\Plasma-state-identification-paper\experiments\..\utils\datasets.py:67: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:204.)
  tensor = torch.tensor([x[i:i+length] for i in range(0, len(x)-(length-stride+1), stride)])
100%|██████████| 20/20 [01:36<00:00,  4.84s/it]


Processing unsupervised set


100%|██████████| 20/20 [01:47<00:00,  5.37s/it]


In [5]:
dataloaders

{'sup': <torch.utils.data.dataloader.DataLoader at 0x200ff33e620>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x200ff33e740>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x200ff33ee00>}

In [6]:
len(dataloaders["sup"])

2683

In [7]:
x,y = next(iter(dataloaders["sup"]))

In [8]:
x.shape, y.shape

(torch.Size([20, 5, 160]), torch.Size([20]))

In [9]:
model = nn.Sequential(
    nn.Conv1d(5, 16, 2),#19
    nn.Conv1d(16,32,3),#17
    nn.Conv1d(32,64,3),#15
    nn.Conv1d(64,128,4),#12
    nn.MaxPool1d(3), #4
    nn.Conv1d(128,128,2), #3
    nn.Conv1d(128,64,2), #2
    nn.AdaptiveAvgPool1d(output_size=1),
    nn.Flatten(),
    nn.Linear(64,4)
)

In [10]:
model

Sequential(
  (0): Conv1d(5, 16, kernel_size=(2,), stride=(1,))
  (1): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (2): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (3): Conv1d(64, 128, kernel_size=(4,), stride=(1,))
  (4): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (5): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (6): Conv1d(128, 64, kernel_size=(2,), stride=(1,))
  (7): AdaptiveAvgPool1d(output_size=1)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=64, out_features=4, bias=True)
)

In [11]:
y.shape, model(x).shape

(torch.Size([20]), torch.Size([20, 4]))

In [12]:
nn.CrossEntropyLoss()(model(x),y)

tensor(1.3530, grad_fn=<NllLossBackward0>)

In [13]:
import utils.inference as inference


In [14]:
x.permute((0,2,1)).shape

torch.Size([20, 160, 5])

In [15]:
f= nn.LSTM(5,4)
f

LSTM(5, 4)

In [16]:
y_pred, hidden = f(x.permute((0,2,1)))

In [17]:
y_pred.shape

torch.Size([20, 160, 4])